In [ ]:
FTP

In [ ]:
Conductivity.json
{
  "ip": "10.143.78.51",
  "limit_time": "2023-12-26 12:01:00",
  "user": "Admin",
  "dst_dir": "A:/020/Conductivity",
  "password": "kyocera",
  "local_dir": "/IoT/bin/ftp/tmp/020/Conductivity"
}

In [ ]:
FTPManager.py
import os
import sys
import json
import ntpath
import datetime
import platform
import posixpath
import ftplib
import logging
import logging.handlers
import re

# 改行コード
linesep = os.linesep
if platform.system() == 'Windows':
    linesep = '\n' #何故かWindowsのCRLFは上手くcsv改行ができない

# アドレスの区切り文字設定
def path_join(*args, os = None):
    os = os if os else platform.system()
    if os == 'Windows':
        return ntpath.join(*args)
    else:
        return posixpath.join(*args)

# JSONファイル読込（コメント付きも対応）
def JSON_load(path_copylog):
    json_obj = None #空データ作成
    try:
        with open(path_copylog, 'r', encoding='utf-8') as f:      # 開く
            text = f.read()                                   # 文字列を取得
        re_text = re.sub(r'/\*[\s\S]*?\*/|//.*', '', text)    # コメントを削除
        json_obj = json.loads(re_text)  
    except Exception as e:
        logger.error("<<error>> json file read")
        logger.error(str(e))
        json_obj=None
    finally:
        return(json_obj)

#**************************************************************************************
#***************************JSONファイルのValueを更新*******************************
#**************************************************************************************
def JSON_change_value(path_json):
    
    try:
        with open(path_json, mode='wt', encoding='utf-8') as file:
            json.dump(j_data, file, ensure_ascii=False, indent=2)
        return True   
    except Exception as e:
        logger.error("<<error>> can't change json file value")
        logger.error(str(e))
        return False
#**************************************************************************************
#**************************************************************************************

#**************************************************************************************
#*******************************FTPサーバー接続*****************************************
#**************************************************************************************
def FTP_connect(ip,user,password,acct=21,timeout=500):
    
    try:
        ftp_conn = ftplib.FTP(ip,user,password,acct,timeout)
        return ftp_conn
    except Exception as e:
        logger.error("<<error>> can't Connect FTP Server")
        logger.error(str(e))
        return None
    
#**************************************************************************************
#**************************************************************************************

#**************************************************************************************
#**********************対象フォルダのファイル一覧を作成**********************************
#**************************************************************************************
def FTP_getfilelist(dst_dir):
    ftp_file = []
    try:      
        ftp_conn.cwd(dst_dir) 
        ftp_conn.dir('./', ftp_file.append)
    except Exception as e:
        logger.error("<<error>> can't get FTP File List")
        logger.error(str(e))
    finally:
        return ftp_file

#**************************************************************************************
#**********************ファイルの更新日時をAM/PM →→→ 24h表記へ変更***********************
#**************************************************************************************
def Change_tm_format(time_ampm, yymmdd):

    # AM/PM表示を24時間表示に変更
    # 0時がAM12:00と表示されるため0:00に修正
    tm_update = None
    try:
        if "PM" in time_ampm:
            tm_update = datetime.datetime.strptime(yymmdd+' '+time_ampm, '%m-%d-%y %H:%M%p') + datetime.timedelta(hours=12)
        elif "AM" and "12" in time_ampm:
            tm_update = datetime.datetime.strptime(yymmdd+' '+time_ampm, '%m-%d-%y %H:%M%p') - datetime.timedelta(hours=12)
        else:
            tm_update = datetime.datetime.strptime(yymmdd+' '+time_ampm, '%m-%d-%y %H:%M%p')
    except Exception as e:
        logger.error("<<error>> can't change time format")
        logger.error(str(e))
    finally:
        return tm_update
#**************************************************************************************
#**************************************************************************************

#**************************************************************************************
#**************limit_timeより更新時間が新しければファイルをダウンロード*******************
#**************************************************************************************
def FTP_getfiles(filepath_ref, filepath_save):

    try:
        with open(filepath_save, 'wb') as download_f: #保存先のファイルを作成
            ftp_conn.retrbinary('RETR %s' % filepath_ref, download_f.write) #参照ファイルを引用、保存
        return True
    
    except Exception as e:
        logger.error("<<error>> can't get file:", filepath_ref, filepath_save)
        logger.error(str(e))
        return False
#************************************************************************************** 
#**************************************************************************************

# グローバル変数
rst = 0 #処理結果(0は処理正常)
dt_st = datetime.datetime.now()
root_dir = path_join("/IoT", "bin") #大本のフォルダを指定
eqp_id = "" #装置ID
eqp_name = "" #装置名
unit_num = "" #ユニットNo
unit_name = "" #ユニット名
log_num = "" #ログNo(QC、TS、hokyuのみ)
target_table_base = "" #登録先テーブル名(FTPは空白)
Dtype = "FTP" #データの種類
ProcType = "FTP" #処理の種類
ftp_conn = None
j_data = None
measured_dt_latest = "" #最終履歴日時

##################################       logファイルの設定　　     #######################################
os.makedirs(path_join(os.path.dirname(__file__) , "log"), exist_ok=True) #ログ保存ディレクトリを作成
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO) #INFO以上は出力
log_Path = path_join(os.path.dirname(__file__) , "log", "debug_"+ dt_st.strftime('%Y%m%d') +".log")
file_handler = logging.FileHandler(log_Path,encoding ='utf-8')
file_handler.setLevel(logging.INFO) #INFO以上は出力
file_handler_format = logging.Formatter('%(asctime)s : %(levelname)s - %(message)s')
file_handler.setFormatter(file_handler_format)
logger.addHandler(file_handler)

logger.info("==========================================================================================================")
logger.info("Program: " + __file__)
logger.info('Proc Start: ' + dt_st.strftime('%Y/%m/%d %H:%M:%S.%f'))
logger.info('----- Python Version -------------------')
logger.info(sys.version)

# jsonファイルの読み込み
try:
    args = sys.argv
    logger.debug("args: " + str(args))

    path_json = args[1]
    j_data = JSON_load(path_json)

    if j_data is None:
        rst = 1 #処理異常
    else:
        ip = j_data["ip"]
        user = j_data["user"]
        password = j_data["password"]
        dst_dir = j_data["dst_dir"]

        local_dir = j_data["local_dir"]
        os.makedirs(local_dir,exist_ok=True) #保存ディレクトリを作成

        limit_time = datetime.datetime.strptime(j_data["limit_time"],"%Y-%m-%d %H:%M:%S")
        measured_dt_latest = limit_time # 最終更新日時を設定

    if len(args) > 2:
        #共通設定ファイルが実装されるまで、暫定で装置・ユニット情報を直接送信
        eqp_id = args[2]
        eqp_name = args[3] 
        unit_num = args[4]
        unit_name = args[5]
        Dtype = args[6]
        
        if len(args)==8:
            log_num = args[7]
        logger.info("************" + eqp_id + " " + unit_num + " " + Dtype + ("" if log_num == "" else ("_" + str(log_num))) + "の書き込み START*******************")

    # # 共通設定からデータ取得
    # # 引数を取得(0:実行プログラムアドレス、1:共通設定ファイルアドレス、2:装置情報No、3:ユニットNo、4:ログNo)
    # j_data_com = JSON_load(args[1])
    # if j_data_com is None:
    #     rst = 1 #処理異常
    # else:
    #     # 装置情報(eqp_data)
    #     eqp = args[2]
    #     eqp_data = j_data_com[eqp]
    #     eqp_id = eqp_data["EQPID"] #装置ID
    #     eqp_name = eqp_data["EQPNAME"] #装置名

    #     # ユニット情報(unit_data)
    #     unit_num = args[3] #ユニットNo
    #     unit_data = eqp_data[unit_num] 
    #     unit_name = unit_data["UNIT_NAME"] #ユニット名
        
    #     # 対象データ情報(target_data)
    #     Dtype = args[4]
    #     if len(args)==6:
    #         log_num = args[5]
    #         target_data = unit_data[Dtype + "_"+ str(log_num)]
    #     else:
    #         target_data = unit_data[Dtype]

    #     # FTP情報(ftp_data)
    #     ftp_data = j_data_com["TP"]
    #     ftp_ip = ftp_data["IP"]
    #     ftp_user = ftp_data["user"]
    #     ftp_pwd = ftp_data["password"]
    #     cfg_path = target_data["config"]["FTPManager"]
    #     local_dir = target_data["ftp_dir"]

    #     j_data = JSON_load(cfg_path)
    #     if j_data is None:
    #         rst = 1 #処理異常
    #     else:  
    #         dst_dir = j_data["dst_dir"]

except Exception as e:
    logger.exception(str(e))
    rst = 1 #処理異常

if rst == 0:

    # FTPサーバー接続
    ftp_conn = FTP_connect(ip,user,password)
    if ftp_conn is not None:
        # カレントフォルダを指定し、保存ファイル一覧を取得
        ftp_file_list = FTP_getfilelist(dst_dir)
        logger.info("ftp file cnt:" + str(len(ftp_file_list)))

        if len(ftp_file_list) == 0:
            rst = -1 #参照ファイルなし

    else:
        rst = 2 # 接続NG

if rst == 0:
       
    list_tm_update =[] #取得完了ファイルの更新日時

    try:
        for line in ftp_file_list:
            
            # データ行を分割
            split_row = line.encode('Latin-1').decode('utf-8').split(maxsplit = 9)

            # 各データを格納
            filename = split_row[-1] #ファイル名
            filebase = os.path.splitext(filename)[0] #ファイル名(拡張子無し)
            exp = os.path.splitext(filename)[1] #拡張子

            time_ampm = split_row[1] #ファイル更新時間(AMPM表記)
            yymmdd = split_row[0] #ファイル更新日付

            tm_update = Change_tm_format(time_ampm, yymmdd) #時刻データをdatetimeに変換

            if tm_update is not None: #時刻データの変換に失敗している場合は処理スキップ
                if tm_update > limit_time: #前回処理時間より後のデータなら処理実行

                    #参照ファイルアドレス
                    filepath_ref = path_join(dst_dir, filename) 

                    # 保存先の年月フォルダを作成
                    root_save = path_join(local_dir, tm_update.strftime("%Y%m"))
                    os.makedirs(root_save, exist_ok=True)

                    # 特定のファイル名のみ末尾に現在時刻を追記
                    if "ALM" in filename or "STS" in filename: 
                        filepath_save = path_join(root_save, filebase + "_" + dt_st.strftime("%Y%m%d_%H%M%S") + exp) # ファイル名末尾に時刻を追加
                    else:
                        filepath_save = path_join(root_save, filename) # 元のファイル名を使用

                    # 保存処理を実行し、成功したら更新日時をリストに保存
                    if FTP_getfiles(filepath_ref, filepath_save):
                        logger.debug("Downloaded files: [" + filename + "]["+ str(tm_update) +"]")
                        list_tm_update.append(tm_update.strftime('%Y-%m-%d %H:%M:%S'))

    except Exception as e:
        logger.error("<<error>> can't download files")
        logger.error(str(e))

    logger.info("get file cnt:" + str(len(list_tm_update)))

    if len(list_tm_update) > 0:
        max_tm_update = max(list_tm_update)
        logger.info("max_tm_update:" + max_tm_update)
        measured_dt_latest = max_tm_update # 最終更新日時を設定
        j_data["limit_time"] = max_tm_update # 最終更新日時を設定
        if JSON_change_value(path_json): #jsonファイルへ保存を実行
            logger.info("limit_time:" + max_tm_update) 
    else:
        rst = -1 #保存ファイルなし

if rst == 0 or rst == -1: #FTP接続が完了している場合はclose実行
    ftp_conn.close()

# End
dt_en = datetime.datetime.now()
logger.info('Proc End: ' + dt_en.strftime('%Y/%m/%d %H:%M:%S.%f'))
dt_diff = dt_en - dt_st
logger.info('Elapsed Time[s]: %d.%06d' % (dt_diff.seconds, dt_diff.microseconds))

#処理履歴を保存(jsonファイル読込NGと登録データなしは記録しない(rst=1以外を記録))
if rst > 1 or rst == 0:
    log_hist = "hist_registered.log"
    msg_rst = ""
    if rst == 2:
        msg_rst = "FTP接続エラー"

    with open(path_join(root_dir, log_hist) , 'a') as file:
        # # 0:処理タイプ, 1:実行プログラム名, 2:装置ID, 3:装置名 4:ユニットNo, 5:ユニット名, 6:logNo, 7:登録データ種類, 8:登録先テーブル名(FTPは空白) 9:処理完了時間, 10:エラーNo, 11:エラー詳細
        # file.write('"' + ProcType + '"' + ',' +
        #            '"' + os.path.basename(__file__) + '"' + ',' +
        #            '"' + eqp_id + '"' + ',' +
        #            '"' + eqp_name + '"' + ',' +
        #            '"' + unit_num + '"' + ',' +
        #            '"' + unit_name + '"' + ',' +
        #            '"' + str(log_num) + '"' + ',' + 
        #            '"' + Dtype + '"' + ',' + 
        #            '"' + target_table_base + '"' + ',' + 
        #            '"' + dt_en.strftime('%Y-%m-%d %H:%M:%S') + '"' + ',' + 
        #            '"' + str(rst) + '"' + ',' + 
        #            '"' + msg_rst + '"' + linesep) 

        # 0:処理タイプ, 1:実行プログラム名, 2:装置ID, 3:装置名 4:ユニットNo, 5:ユニット名, 6:logNo, 7:登録データ種類, 8:登録先テーブル名(FTPは空白) 9:処理完了時間, 10:エラーNo, 11:エラー詳細, 12:最終データ発生日時
        file.write('"' + ProcType + '"' + ',' +
                   '"' + os.path.basename(__file__) + '"' + ',' +
                   '"' + eqp_id + '"' + ',' +
                   '"' + eqp_name + '"' + ',' +
                   '"' + unit_num + '"' + ',' +
                   '"' + unit_name + '"' + ',' +
                   '"' + str(log_num) + '"' + ',' + 
                   '"' + Dtype + '"' + ',' + 
                   '"' + target_table_base + '"' + ',' + 
                   '"' + dt_en.strftime('%Y-%m-%d %H:%M:%S') + '"' + ',' + 
                   '"' + str(rst) + '"' + ',' + 
                   '"' + msg_rst + '"' + ',' + 
                   '"' + measured_dt_latest + '"' + linesep)
        
# 変数の解放
del ftp_conn
del j_data

if rst != 1:
    logger.info("************" + eqp_id + " " + unit_num + " " + Dtype + ("" if log_num == "" else ("_" + str(log_num))) + "の書き込み END*******************")

logger.info("==========================================================================================================" + linesep)


In [ ]:
Ftp_GetLogs.sh
#!/bin/bash

CURR_DIR=/IoT/bin/ftp
PYSCRIPT=$CURR_DIR/FTPManager.py
SHNAME=$(basename "$0")
CONFIG_DIR=$CURR_DIR/config

#履歴登録(FTP操作実績)
PROC_TYPE=FTP
TIME=`date "+%Y-%m-%d %H:%M:%S"`
RESULT=0
NULL=\"\"
/IoT/bin/hist_append.sh ${PROC_TYPE} ${SHNAME} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} "${TIME}" ${RESULT} ${NULL}

cd $CURR_DIR

# 多重起動防止

SHNAME=$(basename "$0")
LCKNAME=/$SHNAME.lck
if [ -e $CURR_DIR/$LCKNAME ]; then
    exit 0
fi

touch $CURR_DIR/$LCKNAME

EQPID=LEV10001
EQPNAME=酸アルカリ洗浄

UNIT_NUM=000
UNIT_NAME=LINE
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS

UNIT_NUM=010
UNIT_NAME=LD
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME ALM
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS_raw
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/COUNT.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME COUNT

UNIT_NUM=020
UNIT_NAME="酸・アルカリ洗浄"
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME ALM
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS_raw
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/log1_QC.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME QC 1
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/log1_TS.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME TS 1
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/Conductivity.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME Conductivity

UNIT_NUM=030
UNIT_NAME=ULD
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME ALM
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME STS_raw
python3 $PYSCRIPT $CONFIG_DIR/$UNIT_NUM/COUNT.json $EQPID $EQPNAME $UNIT_NUM $UNIT_NAME COUNT

rm -f $CURR_DIR/$LCKNAME


In [ ]:
DB

In [ ]:
Conductivity.json
{
  "comment": "",
  "unit_num": "020",
  "pw": "asd23fgh",
  "eqp_id": "LEV10001",
  "unit_name": "酸・アルカリ洗浄",
  "dst_dir": "/IoT/bin/ftp/tmp/020/Conductivity/*/*CSV",
  "database": "AYB_APPL",
  "username": "IOT_DATA",
  "server": "10.143.16.244",
  "schema": "IOT_DATA",
  "table": "CONDUCTIVITY"
}

In [ ]:
db2_CNDT.py

In [ ]:
import platform
import sys
import glob
import json
from pathlib import Path
import logging
import logging.handlers
import pandas as pd
import ibm_db
import datetime
import os
import csv

################################################################################################################

#定数
max_sqlstr_byte = 2097152 #SQL構文の最大バイト長(DB2の場合)、INSERT等の構文はこのバイト長を超える文字列だとNGになる

# 改行コード
linesep = os.linesep
if platform.system() == 'Windows':
    linesep = '\n' #何故かWindowsのCRLFは上手くcsv改行ができない

rst = 0 #処理結果(0は処理正常)
dt_st = datetime.datetime.now()


##################################       logファイルの設定　　     #######################################

# logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO) #INFO以上は出力
file_handler = logging.FileHandler(os.path.dirname(__file__) + "/debug.log")
# file_handler = logging.FileHandler(os.path.join(os.path.dirname(__file__), "log", "debug_" + datetime.now().strftime('%Y%m%d') + ".log"))

file_handler.setLevel(logging.INFO) #INFO以上は出力
file_handler_format = logging.Formatter('%(asctime)s : %(levelname)s - %(message)s')
file_handler.setFormatter(file_handler_format)
logger.addHandler(file_handler)

logger.info("==========================================================================================================")
logger.info("Program: " + __file__)
logger.info('Proc Start: ' + dt_st.strftime('%Y/%m/%d %H:%M:%S.%f'))
logger.info('----- Python Version -------------------')
logger.info(sys.version)

#############################################################################################################

##################################       設定ファイルの読み込み　　     #######################################
files_ReadErr = [] #過去に読み取りNGで処理されたファイルリスト
Dtype = "Conductivity" #処理データの種類

if rst == 0:
    try:
        args = sys.argv
        # fd = open(args[1], mode='r', encoding='utf-8')
        fd = open(args[1], mode='r')
        # fd = open( "Conductivity.json", mode='r')
        j_data = json.load(fd)
        fd.close()
        logger.info("************" + j_data["unit_num"] + "  " +j_data["unit_name"] + " " + Dtype + "の書き込み START*******************")

        # 対象スキーマ、対象テーブルを取得
        target_schema = j_data["schema"] #スキーマ名(ドットまで記載、指定なければ空文字列)
        target_table_base = j_data["table"] #対象テーブル(一時テーブルも同じ名称から作成する)

        # テーブル名を設定情報を元に作成
        temp_table = "tmp_" + target_table_base + "_" + j_data["eqp_id"] + "_" + j_data["unit_num"] #「temp_対象テーブル_装置ID_ユニットNo」の構成で一時テーブルを設定(一時テーブルの参照がかぶらないように固有の値とする)
        if target_schema == "":
            target_table = target_table_base # Merge対象のテーブル名
        else:
            target_table = target_schema + '.' + target_table_base # Merge対象のテーブル名

        # 読み込みエラーが発生したファイルの履歴ファイル保存フォルダを指定、なければ作成
        # dir_readerr = os.path.dirname(os.path.join(os.getcwd()) + "/ReadErr/" + Dtype)
        dir_readerr = os.path.dirname(__file__) + "/ReadErr/" + Dtype
        if os.path.isdir(dir_readerr) == False:
            os.makedirs(dir_readerr)

        # 読み込みエラー履歴がある場合はNGファイルアドレスをリストで取得
        path_readerr = dir_readerr + "/" + j_data["eqp_id"] + "_" + j_data["unit_num"] + ".csv" #読み込みNGのファイルアドレス保存csv
        if os.path.isfile(path_readerr)==True: #既にファイルが作成済み(過去に読み取りエラーが発生したファイルがある)
            with open(path_readerr, 'r') as file:
                reader = csv.reader(file)
                next(reader) #先頭行をスキップ(データ名を記載しているため)
                for row in reader:
                    files_ReadErr.append(row[0]) # 0:読み取りNGファイルアドレス           

        # 参照フォルダが存在するかを確認、参照フォルダが存在しない（アドレスエラー、または保存対象外）場合はrst=1を指定し、以降の処理を行わない。
        path_dst_dir = j_data["dst_dir"]
        dir_list = glob.glob(os.path.dirname(path_dst_dir))
         # if os.path.isdir(os.path.dirname(path_dst_dir)) == False:
        if not dir_list:  
            logger.error("NG: directory path not exist")
            logger.error("dst_dir: " + path_dst_dir)
            rst = 1 #処理NG

    except Exception as e:
        logger.exception(str(e))
        rst = 1 #処理異常
#############################################################################################################

files_NG = [] #INSERTに失敗したファイルリスト
files_OK = [] #INSERTに成功したファイルリスト
list_sql = [] #実行するSQL構文と一括登録するレコード数、ファイルアドレスのリスト(0:SQL構文、1:登録レコード数, 2:ファイルアドレス)

# 一時テーブルに登録するデータのINSERT構文をcsvファイルを元に作成
if rst == 0:

    dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)
    logger.info("-----   データファイル参照   -----")
    #INSERT対象の列名
    column_insert = ['EQP_ID', 'UNIT_NUM', 'DATE_TIME', 'COND_VALUES']

    
    # json設定ファイルから値を登録する対象列と項目名
    column_add_json = [["EQP_ID", "eqp_id"], 
                      ["UNIT_NUM", "unit_num"]]

    #csvファイルのリスト取得
    files = glob.glob(path_dst_dir)
    logger.info("file cnt:" + str(len(files)))

    #ファイルの数だけループ
    for filepath in files:

        logger.debug("filepath:" + filepath)

        if filepath in files_ReadErr: #過去に読み取り失敗したファイルなら処理しない
            logger.error("Pass: This file was ReadError")
            logger.error("filepath:" + filepath) 
            continue

        # 前回読み取り出来ている、または初回読込ファイルなら処理を継続
        sqlcnt = 0 #1ファイルで作成したSQL構文数

        sql_insert = "" #INSERTのSQL構文
        sql_insert_value = "" #Values部分の構文
        cnt_insert = 0 #Insertレコード数(一括処理するので、登録行数をLogに保存)

        try:
            ################################# データに"装置ID","ユニット名"等追加　######################################
            df_csv = pd.read_csv(filepath,skiprows =12, encoding="cp932", dtype=object)

            # 取得したデータの行数が0の場合は飛ばして次のファイルを参照
            if len(df_csv) == 0:
                logger.error("Pass: This file not exist datarow") 
                logger.error("filepath:" + filepath)
                continue
                
            # ~~~~~~~~~~~~~~ここからcsvデータに合わせた処理を記載~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

            #処理開始時刻の文字列データ(処理基準値として使用)
            dt_st_str = dt_st.strftime("%Y/%m/%d %H:%M:%S")

            # 新しいデータフレームに必要な情報を抜粋してゆく
            df_buff =pd.DataFrame({})

            df_buff['DATE_TIME'] = (pd.to_datetime(df_csv[':DEV_SIZE'], format='%Y/%m/%d %H:%M:%S')).dt.strftime('%Y-%m-%d %H:%M:%S') #DB2登録用のDate文字列を設定
           
            df_buff['COND_VALUES'] = df_csv["1"].astype(float) * 0.09 / 3780
   
            # json設定ファイルからデータを引用、保存
            for colname, jsonid in column_add_json:
                df_buff[colname] = j_data[jsonid]

            # 列順再設定用の列名リストを作成(今回は追加列がないのでcolumn_insertを使用）
            column_reindex = column_insert

            # ~~~~~~~~~~~~~~ csv処理の終了 ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

            # 列順を変更(sql_insertに合わせる)
            df_buff = df_buff.reindex(columns = column_reindex)

            # 全データのデータ型を再度文字列に変更
            df_buff = df_buff.astype(str)

            # 欠損値(NaN)を文字列「NULL」に置換(念のために設定)
            df_buff = df_buff.replace('nan' ,'NULL')

            # 列名をカンマ区切りで文字列結合
            str_columnname = ','.join(column_reindex)

            # Value構文用にカンマ区切りで結合したリストを作成
            df_insert = pd.DataFrame({})

            for column_name in column_reindex:
                if column_name == column_reindex[0]:
                    df_cat = "'" + df_buff[column_name].astype(str) + "'" #一旦クオーテーションを付けたデータを作成
                    df_cat = df_cat.replace("'NULL'" ,'NULL') #NULLだけクオーテーション無しに設定(クオーテーションありだと文字列として登録されるため)
                    df_insert['Values'] = df_cat  #??????
                else:
                    df_cat = "'" + df_buff[column_name].astype(str) + "'" #一旦クオーテーションを付けたデータを作成
                    df_cat = df_cat.replace("'NULL'" ,'NULL') #NULLだけクオーテーション無しに設定(クオーテーションありだと文字列として登録されるため)
                    df_insert['Values'] = df_insert['Values'].str.cat(df_cat, sep=',') #各列をカンマ区切りで結合

            # dataframe列をリストに変換(処理を早くするため)
            list_insert = '(' + df_insert['Values'] + ')'

            # INSERTの基本構文を作成(列順はDataframe準拠。処理は後述の登録テーブルへのMERGEと同等、但し登録先は一時テーブル、登録データはデータファイルのValuesを結合したものを使用)           
            # 登録先を指定(一時テーブルが対象)
            sql_insert_top = "MERGE INTO " + temp_table + " AS TGT USING (VALUES "
            
            # データ元を指定(データファイルのValuesを結合したものを疑似テーブルとして使用し、カラム名を設定)
            sql_insert_buttom = ") AS INSERTDATA (" + ','.join([datarow for datarow in column_reindex]) + ")"

            # どのカラムを条件にINSERT or UPDATEを実行するか指定
            sql_insert_under = " ON ("\
                                " TGT.EQP_ID = INSERTDATA.EQP_ID"\
                                " AND TGT.UNIT_NUM = INSERTDATA.UNIT_NUM"\
                                " AND TGT.DATE_TIME = INSERTDATA.DATE_TIME"\
                            ") "\
                            "WHEN MATCHED THEN "\
                                "UPDATE SET "\
                                "TGT.COND_VALUES = INSERTDATA.COND_VALUES "\
                            "WHEN NOT MATCHED THEN"

            
            sql_insert_under += ' INSERT (' + ','.join([datarow for datarow in column_reindex]) + ')'
            sql_insert_under += ' VALUES (INSERTDATA.' + ',INSERTDATA.'.join([datarow for datarow in column_reindex]) + ');' #????
            #############################################################################################################

            ###############################   1行ずつ処理、SQL構文を作成し保存　　##########################################
            for str_value in list_insert:

                flg_append = True #list_sqlへ追加するかの判定(Trueで追加実行)

                # INSERT構文を作成(SQL構文をバイト数に換算した場合の最大制限があるので、それ以下まで行ごとにValueを追記する)
                if len((sql_insert_top + sql_insert_value + "," + str_value + "," + str_value + sql_insert_buttom + sql_insert_under).encode('utf-8')) >= max_sqlstr_byte:
                    #1. 文字列のバイト制限ギリギリになったらInsert実行(次回のValue追加で制限超えるのを疑似的に判定するため、同じValueを重ねて判定)
                    sql_insert = sql_insert_top + sql_insert_value + "," + str_value + sql_insert_buttom + sql_insert_under
                    cnt_insert += 1 # 登録レコード数を+1
                    list_sql.append([sql_insert, cnt_insert, filepath]) #SQLリストにSQL構文とレコード数、ファイルアドレスを登録
                    logger.debug("sql_cnt:" + str(cnt_insert) + ", sql_insert:" + sql_insert)

                    sql_insert_value = "" #Value文字列をリセット
                    cnt_insert = 0 #レコード数をリセット
                    flg_append = False #登録しない(この処理を最後にループを抜けた場合はリスト登録は行われない)
                    sqlcnt += 1 #作成構文数を+1

                else:
                    #2. バイト制限まで余裕がある場合は、Valueだけ結合
                    if sql_insert_value == "":
                        sql_insert_value = str_value
                    else:
                        sql_insert_value += "," + str_value
                    cnt_insert += 1 # 登録レコード数を+1

            if flg_append == True:
                sql_insert = sql_insert_top + sql_insert_value + sql_insert_buttom + sql_insert_under
                list_sql.append([sql_insert, cnt_insert, filepath]) #SQLリストにSQL構文とレコード数、ファイルアドレスを登録
                logger.debug("sql_cnt:" + str(cnt_insert) + ", sql_insert:" + sql_insert)
                sqlcnt += 1 #作成構文数を+1

            logger.debug("sqlcnt:" + str(sqlcnt))

            ##############################################################################################################

        except Exception as e:
            logger.exception(str(e))
            files_NG.append(filepath) #登録NGのファイルアドレスを追加
            #rst = 3 #SQL処理NG

    # 登録するデータがない（ファイルがない、または取得できたデータがない）場合はrst=1を指定し、以降の処理を行わない
    if len(list_sql) == 0:
        logger.error("NG: insert data is nothing")
        rst = 1 # 処理NG

    #処理時間(デバッグ用)
    dt_en_process = datetime.datetime.now()
    dt_diff_process = dt_en_process - dt_st_process
    logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))
    
##################################              DB接続                #######################################
if rst == 0:
    
    dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)

    logger.info("-----   DB接続実行   -----")
    try:
        server = j_data["server"]
        database = j_data["database"]
        username = j_data["username"]
        password = j_data["pw"]
        conn = ibm_db.connect("DATABASE=" + database + ";HOSTNAME=" + server + ";PORT=50000;PROTOCOL=TCPIP;UID=" + username + ";PWD=" + password + ";", "", "")
        logger.debug('OK')

    except Exception as e:
        logger.error('NG')
        logger.exception(str(e))
        rst = 2 #DB接続NG

    #処理時間(デバッグ用)
    dt_en_process = datetime.datetime.now()
    dt_diff_process = dt_en_process - dt_st_process
    logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))

##########################################################################################################

############################   登録テーブルのカラム構造を取得   #############################################
if rst == 0:
    
    dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)

    logger.info("-----   登録テーブル構造取得   -----")
    try:

        # 対象テーブルのカラム構成(列名、データ型、データ長を取得)
        sql_select = 'SELECT NAME, COLTYPE, LENGTH FROM SYSIBM.SYSCOLUMNS'\
                    ' WHERE TBNAME = ' + "'" + target_table_base + "'"\
                    ' AND TBCREATOR = ' + "'" + target_schema + "'"\
                    ' ORDER BY COLNO'
        rstset = ibm_db.exec_immediate(conn, sql_select) #create temp table実行 
        if rstset is False:
            logger.error("CNT:NG")
        else:
            logger.debug("CNT:OK")
            colname_target = []
            datarow = ibm_db.fetch_tuple(rstset)
            while datarow:
                colname_target.append([datarow[0], datarow[1], datarow[2]])
                datarow = ibm_db.fetch_tuple(rstset)
            logger.debug(colname_target)

    except Exception as e:
        logger.exception(str(e))
        rst = 3 #一時テーブル作成NG

    #処理時間(デバッグ用)
    dt_en_process = datetime.datetime.now()
    dt_diff_process = dt_en_process - dt_st_process
    logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))

#############################################################################################################

##################################   一時テーブルを作成   ##################################################

if rst == 0:
    dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)

    logger.info("-----   TEMP TABLE作成実行   -----")
    try:
        # インデックス設定の対象列名、一時テーブルに設定
        column_index_tmp = ['EQP_ID', 'UNIT_NUM', 'DATE_TIME', 'COND_VALUES']

        # 列名とデータ型の構成リスト、先述の対象テーブルのカラム構成をそのまま使って一時テーブルを作成
        column_create = []
        for datarow in colname_target:
            colname = datarow[0]
            coltype = datarow[1].replace(' ', '') #スペースが入ることがあるので削除する
            collength = datarow[2] 

            # データ型とデータ長を用いてを設定用データ型に変換
            if 'VAR' in coltype or 'CHAR' in coltype:
                column_create.append([colname, coltype + '(' + str(collength) + ')']) #設定長を含めてデータ型を設定
            elif 'TIMESTMP' == coltype:
                column_create.append([colname, 'TIMESTAMP']) #返り値はTIMESTMPになるので、TIMESTAMPで設定(※SYSTEM側から返ってくるデータ型はクライアント指定のと違う場合があるので注意)
            elif 'DECIMAL' == coltype:
                column_create.append([colname, coltype + '(6,5)']) 
            else:
                column_create.append([colname, coltype]) #データ型をそのまま設定

        # CREATEで一時テーブル(TEMP)を作成
        sql_create = "CREATE TEMP TABLE " + temp_table + " ("
        sql_create += ','.join([clmlist[0] + ' ' + clmlist[1] for clmlist in column_create]) + ");" # 「列名 データ型」の文字列をカンマ区切りで結合
        rstset = ibm_db.exec_immediate(conn, sql_create) #create temp table実行 
        if rstset is False:
            print('CREATE:NG')
            logger.error("CREATE:NG")
            rst = 3 #一時テーブル作成NG
        else:
            logger.debug("CREATE:OK")
            # インデックス作成を実行
            sql_create = "CREATE INDEX IDX_" + temp_table + " ON " + temp_table + "(" + ','.join([datarow for datarow in column_index_tmp]) + ");"
            rstset = ibm_db.exec_immediate(conn, sql_create) #create temp table実行 

            if rstset is False:
                logger.error("INDEX:NG")
                # rst = 3 #一時テーブル作成NG
            else:
                logger.debug("INDEX:OK")

    except Exception as e:
        logger.exception(str(e))
        rst = 3 #一時テーブル作成NG

    #処理時間(デバッグ用)
    dt_en_process = datetime.datetime.now()
    dt_diff_process = dt_en_process - dt_st_process
    logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))

#############################################################################################################


insertcnt = 0 #TEMP TABLEへINSERT出来たレコード数
if rst == 0:
    if len(list_sql) > 0: #実行するSQL構文がある場合
        
        ###############################   SQL構文を一時テーブルに処理実行　　##########################################

        dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)

        logger.info("-----   INSERT処理実行   -----")
        for sql_insert , cnt_insert , filepath in list_sql:

            if filepath not in files_NG:
                
                # INSERT失敗したファイルのSQL構文でなければINSERT実行
                logger.debug("insert cnt:" + str(cnt_insert))
                logger.debug("insert sql:" + sql_insert[:200] + "……")
                logger.debug("insert filepath:" + filepath)

                #該当データがない場合(cnt_insert=0)は処理を飛ばす
                if cnt_insert == 0: 
                    logger.debug("Pass: insert cnt = 0")  
                    continue
                
                #登録するValueが取得出来ている場合は処理継続
                try:
                    dt_st_sql = datetime.datetime.now() #SQL開始時間(デバッグ用)
                    rstset = ibm_db.exec_immediate(conn, sql_insert) #Insert実行
                    dt_en_sql = datetime.datetime.now() #SQL終了時間(デバッグ用)
                    if rstset is False:
                        logger.error("NG")
                        dt_diff_sql = dt_en_sql - dt_st_sql
                        logger.debug('SQL Time[s]: %d.%06d' % (dt_diff_sql.seconds, dt_diff_sql.microseconds))
                        
                    else:
                        logger.debug("OK")
                        dt_diff_sql = dt_en_sql - dt_st_sql
                        logger.debug('SQL Time[s]: %d.%06d' % (dt_diff_sql.seconds, dt_diff_sql.microseconds))
                        files_OK.append(filepath) #全SQL成功したらOKファイルリスト追加
                        files_OK = sorted(files_OK) #重複削除

                except Exception as e:
                    logger.error("NG")
                    logger.exception(str(e))
                    if not filepath in files_NG:
                        files_NG.append(filepath) #登録NGのファイルアドレスを追加
                    files_NG = sorted(files_NG) #重複削除
                    if len(files_OK) > 0:
                        if filepath in files_OK:
                            files_OK.remove(filepath) #登録OKのファイルアドレスを削除

        try:
            sql_select ="SELECT COUNT(*) AS CNT FROM " + temp_table
            dt_st_sql = datetime.datetime.now() #SQL開始時間(デバッグ用)
            rstset = ibm_db.exec_immediate(conn, sql_select) #SELECT CNT実行
            dt_en_sql = datetime.datetime.now() #SQL終了時間(デバッグ用)
            if rstset is False:
                logger.error("CNT:NG")
                dt_diff_sql = dt_en_sql - dt_st_sql
                logger.debug('SQL Time[s]: %d.%06d' % (dt_diff_sql.seconds, dt_diff_sql.microseconds))
            else:
                logger.debug("CNT:OK")
                dt_diff_sql = dt_en_sql - dt_st_sql
                logger.debug('SQL Time[s]: %d.%06d' % (dt_diff_sql.seconds, dt_diff_sql.microseconds))
                datarow = ibm_db.fetch_tuple(rstset)
                insertcnt = datarow[0] #CNTを取得
                logger.debug("recordcnt:" + str(insertcnt))
                
        except Exception as e:
            logger.error("CNT ERROR")
            logger.exception(str(e))

        #処理時間(デバッグ用)
        dt_en_process = datetime.datetime.now()
        dt_diff_process = dt_en_process - dt_st_process
        logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))

    else:
        logger.exception("登録SQL数:0")
        rst = 4 #一時テーブル作成NG
        ##############################################################################################################

    # 登録OKのファイル0の場合（レコードなし、または途中のデータが登録不可で処理中断)
    if len(files_OK) == 0:
        rst = 4 #一時テーブルにレコード未登録



if rst == 0 and insertcnt > 0:
    ##########################   一時テーブルのデータを登録テーブルにMERGE   ########################################
    
    dt_st_process = datetime.datetime.now() #処理開始時間(デバッグ用)

    logger.info("-----   MERGE処理実行   -----")
    try:

        sql_merge = "MERGE INTO " + target_table + " AS TGT "\
                    "USING " + temp_table + " AS TMP "\
                    "ON ("\
                        "TGT.EQP_ID = TMP.EQP_ID"\
                        " AND TGT.UNIT_NUM = TMP.UNIT_NUM"\
                        " AND TGT.DATE_TIME = TMP.DATE_TIME"\
                    ") "\
                    "WHEN MATCHED THEN "\
                        "UPDATE SET "\
                        "TGT.COND_VALUES = TMP.COND_VALUES "\
                    "WHEN NOT MATCHED THEN"

        sql_merge += ' INSERT (' + ','.join([datarow[0] for datarow in colname_target]) + ')'
        sql_merge += ' VALUES (TMP.' + ',TMP.'.join([datarow[0] for datarow in colname_target]) + ');' #登録テーブルのカラムを全対象にしてINSERT(一時テーブルも同一構成なので、そのまま登録できる)

        #MERGE処理を実行
        logger.debug("SQL_merge:" + sql_merge)
        rstset = ibm_db.exec_immediate(conn, sql_merge)
        if rstset is False:
            logger.error("NG")
            rst = 5 #MERGE処理NG
        else:
            logger.debug("OK")

    except Exception as e:
        logger.exception(str(e))
        rst = 5 #MERGE処理NG
        files_NG = files #全ファイルの登録が出来ていないので、NGリストを全ファイルに設定

    #処理時間(デバッグ用)
    dt_en_process = datetime.datetime.now()
    dt_diff_process = dt_en_process - dt_st_process
    logger.debug('Processed Time[s]: %d.%06d' % (dt_diff_process.seconds, dt_diff_process.microseconds))

    ##############################################################################################################
# 一時テーブルまで作成済みなら一時テーブル削除(無くてもいいが一応実行)
if rst == 0 or rst >= 3:
    ##########################             一時テーブルの削除              ########################################
    try:
        logger.info("-----   TEMP TABLE削除   -----")
        sql_drop = "DROP TABLE " + temp_table +" ;"
        stmt = ibm_db.exec_immediate(conn, sql_drop)
        logger.debug("OK")
    except Exception as e:
        logger.error("NG")
        logger.exception(str(e))
    
    ##############################################################################################################

    ##########################                DB2接続終了                 ########################################
    try:
        logger.info("-----   DB接続終了   -----")
        ibm_db.close(conn) #接続終了
        logger.debug("OK")
    except Exception as e:
        logger.error("NG")
        logger.exception(str(e))

    ##############################################################################################################


   ##########################            登録NGファイル処理              ########################################
try:
    if len(files_NG) > 0:
        # 登録NGのファイルアドレスを指定csvに追記
        with open(path_readerr, 'a') as file:
            for filepath in files_NG:
                file.write('"' + filepath + '"' + linesep) # 0:ロットNo, 1:着手時間        

except Exception as e:
    logger.error("SQL Execution Error")
    logger.exception(str(e))
    logger.error("Error SQL: " + sql_merge[:200] + "……")  # Log the SQL statement causing the error

   ##############################################################################################################

# End
dt_en = datetime.datetime.now()
logger.info('Proc End: ' + dt_en.strftime('%Y/%m/%d %H:%M:%S.%f'))
dt_diff = dt_en - dt_st
logger.info('Elapsed Time[s]: %d.%06d' % (dt_diff.seconds, dt_diff.microseconds))
logger.info("************" + j_data["unit_num"] + "  " + j_data["unit_name"] + " " + Dtype + "の書き込み END*********************")
logger.info("==========================================================================================================")

In [ ]:
DB_insert.sh
#!/bin/bash

CURR_DIR=/IoT/bin/DB
PY_STS=$CURR_DIR/db2_STS_r1.py
PY_STS_RAW=$CURR_DIR/db2_STS_raw_r1.py
PY_ALM=$CURR_DIR/db2_ALM_r1.py
PY_QC=$CURR_DIR/db2_QC_r1.py
PY_TS=$CURR_DIR/db2_TS_r1.py
PY_COUNT=$CURR_DIR/db2_COUNT_r1.py
PY_HOKYU=$CURR_DIR/db2_hokyu_r1.py
PY_CNDT=$CURR_DIR/db2_CNDT.py
SHNAME=$(basename "$0")
LCKNAME=/$SHNAME.lck
CONFIG_DIR=$CURR_DIR/config

#履歴登録(DB登録開始)
PROC_TYPE=DB
TIME=`date "+%Y-%m-%d %H:%M:%S"`
RESULT=0
NULL=\"\"
/IoT/bin/hist_append.sh ${PROC_TYPE} ${SHNAME} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} "${TIME}" ${RESULT} ${NULL}

# 多重起動防止
if [ -e $CURR_DIR/$LCKNAME ]; then
    exit 0
fi

touch $CURR_DIR/$LCKNAME

#DB2環境変数設定
cd /IoT/db2driver/dsdriver
. ./db2profile

#　PC設備のデータをDBへ書き込む
cd $CURR_DIR

EQPID=LEV10001
EQPNAME=酸アルカリ洗浄

# 000
UNIT_NUM=000
python3 $PY_STS $CONFIG_DIR/$UNIT_NUM/STS.json $EQPNAME

# 010
UNIT_NUM=010
python3 $PY_ALM $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPNAME
python3 $PY_STS $CONFIG_DIR/$UNIT_NUM/STS.json $EQPNAME
python3 $PY_STS_RAW $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPNAME
python3 $PY_COUNT $CONFIG_DIR/$UNIT_NUM/COUNT.json $EQPNAME

# 020
UNIT_NUM=020
python3 $PY_STS $CONFIG_DIR/$UNIT_NUM/STS.json $EQPNAME
python3 $PY_STS_RAW $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPNAME
python3 $PY_ALM $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPNAME
python3 $PY_TS $CONFIG_DIR/$UNIT_NUM/log1_TS.json $EQPNAME
python3 $PY_QC $CONFIG_DIR/$UNIT_NUM/log1_QC.json $EQPNAME
python3 $PY_CNDT $CONFIG_DIR/$UNIT_NUM/Conductivity.json $EQPNAME

# 030
UNIT_NUM=030
python3 $PY_ALM $CONFIG_DIR/$UNIT_NUM/ALM.json $EQPNAME
python3 $PY_STS $CONFIG_DIR/$UNIT_NUM/STS.json $EQPNAME
python3 $PY_STS_RAW $CONFIG_DIR/$UNIT_NUM/STS_raw.json $EQPNAME
python3 $PY_COUNT $CONFIG_DIR/$UNIT_NUM/COUNT.json $EQPNAME

rm $CURR_DIR/$LCKNAME

In [ ]:
CopyLogs.sh

In [ ]:
#!/bin/bash

# 多重起動防止
SHNAME=$(basename "$0")
LCKNAME=/$SHNAME.lck
CURR_DIR=/IoT/bin

#履歴登録(NAS操作実績)
PROC_TYPE=NAS
TIME=`date "+%Y-%m-%d %H:%M:%S"`
RESULT=0
NULL=\"\"
/IoT/bin/hist_append.sh ${PROC_TYPE} ${SHNAME} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} "${TIME}" ${RESULT} ${NULL}

if [ -e $CURR_DIR/$LCKNAME ]; then
    exit 0
fi

touch $CURR_DIR/$LCKNAME

cd $CURR_DIR

# NAS　umount確認
sudo /bin/umount /IoT/bin/nas
#NAS
sudo /bin/mount -t cifs -o vers=2.0,domain='AD',user='Ayabe3user',password='Zaq12wsx' //10.143.16.193/01_Data/Bldg3/B35_LEV10001 /IoT/bin/nas


# データ収集対象 tmp/ -> nas
# A && B：Aが成功したらBを実行
# 　option : -r ディレクトリ　　-u タイムスタンプが新しいファイルのみコピー
#PLC設備　----→　NAS
sudo cp -r -u /IoT/bin/ftp/tmp/000 /IoT/bin/nas && sudo cp -r -u /IoT/bin/ftp/tmp/000 /IoT/bin/swap && rm -r /IoT/bin/ftp/tmp/000/*
sudo cp -r -u /IoT/bin/ftp/tmp/010 /IoT/bin/nas && sudo cp -r -u /IoT/bin/ftp/tmp/010 /IoT/bin/swap && rm -r /IoT/bin/ftp/tmp/010/*
sudo cp -r -u /IoT/bin/ftp/tmp/020 /IoT/bin/nas && sudo cp -r -u /IoT/bin/ftp/tmp/020 /IoT/bin/swap && rm -r /IoT/bin/ftp/tmp/020/*
sudo cp -r -u /IoT/bin/ftp/tmp/030 /IoT/bin/nas && sudo cp -r -u /IoT/bin/ftp/tmp/030 /IoT/bin/swap && rm -r /IoT/bin/ftp/tmp/030/*

#umount
sudo /bin/umount /IoT/bin/nas
# ロックファイル削除
rm $CURR_DIR/$LCKNAME

In [ ]:
GetLogs.sh

In [ ]:
#!/bin/bash

# 多重起動防止
SHNAME=$(basename "$0")
LCKNAME=/$SHNAME.lck
CURR_DIR=/IoT/bin

#履歴登録(PC起動実績)
PROC_TYPE=PC
TIME=`date "+%Y-%m-%d %H:%M:%S"`
RESULT=0
NULL=\"\"
$CURR_DIR/hist_append.sh ${PROC_TYPE} ${SHNAME} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} ${NULL} "${TIME}" ${RESULT} ${NULL}

if [ -e $CURR_DIR/$LCKNAME ]; then
    exit 0
fi

touch $CURR_DIR/$LCKNAME

# $CURR_DIR/smb/MountLogs.sh

$CURR_DIR/ftp/Ftp_GetLogs.sh

$CURR_DIR/date_judge.sh
if [ $? -eq 0 ]; then

$CURR_DIR/DB/DB_insert.sh

$CURR_DIR/CopyLogs.sh

$CURR_DIR/Registered_Hist.sh

fi

rm $CURR_DIR/$LCKNAME


In [ ]:
# アドレスの区切り文字設定
def path_join(*args, os = None):
    os = os if os else platform.system()
    if os == 'Windows':
        return ntpath.join(*args)
    else:
        return posixpath.join(*args)


rst = 0 #処理結果(0は処理正常)
dt_st = datetime.datetime.now()


##################################       logファイルの設定　　     #######################################

os.makedirs(path_join(os.path.dirname(__file__) , "log"), exist_ok=True) 

logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)

log_Path = path_join(os.path.dirname(__file__) , "log", "debug_"+ dt_st.strftime('%Y%m%d') +".log")
file_handler = logging.FileHandler(log_Path,encoding ='utf-8')
file_handler.setLevel(logging.INFO) 

file_handler_format = logging.Formatter('%(asctime)s : %(levelname)s - %(message)s')
file_handler.setFormatter(file_handler_format)
logger.addHandler(file_handler)

logger.info("==========================================================================================================")
logger.info("Program: " + __file__)

logger.info('Proc Start: ' + dt_st.strftime('%Y/%m/%d %H:%M:%S.%f'))
logger.info('----- Python Version -------------------')
logger.info(sys.version)

#############################################################################################################

In [ ]:
import os
import logging
import sys
from os.path import join as path_join
from datetime import datetime

# Create a directory for storing logs, and if it already exists, do nothing
os.makedirs(path_join(os.path.dirname(__file__), "log"), exist_ok=True)

# Get the logger for the current module (__name__)
logger = logging.getLogger(__name__)

# Set the logging level to INFO, meaning it will log messages with INFO level and above
logger.setLevel(logging.INFO)

# Define the path for the log file based on the current file's directory, 'log' subdirectory,
# and a filename that includes the current date in the format 'debug_YYYYMMDD.log'
log_Path = path_join(os.path.dirname(__file__), "log", "debug_" + datetime.now().strftime('%Y%m%d') + ".log")

# Create a file handler for logging to the specified log file, using UTF-8 encoding
file_handler = logging.FileHandler(log_Path, encoding='utf-8')

# Set the logging level for the file handler to INFO
file_handler.setLevel(logging.INFO)

# Define the log message format with timestamp, log level, and message
file_handler_format = logging.Formatter('%(asctime)s : %(levelname)s - %(message)s')

# Set the formatter for the file handler
file_handler.setFormatter(file_handler_format)

# Add the file handler to the logger
logger.addHandler(file_handler)

# Log separator and additional information at the beginning of the log file
logger.info("==========================================================================================================")
logger.info("Program: " + __file__)

# Log the start time of the program
logger.info('Proc Start: ' + datetime.now().strftime('%Y/%m/%d %H:%M:%S.%f'))

# Log the Python version information
logger.info('----- Python Version -------------------')
logger.info(sys.version)

In [ ]:
import csv
import platform
import sys
import glob
import json
from pathlib import Path
import logging
import logging.handlers
import pandas as pd
import ibm_db
import datetime
import os
import ntpath
import posixpath
import shutil

In [ ]:
#処理履歴を保存(jsonファイル読込NGと登録データなしは記録しない(rst=1以外を記録))
if rst > 1 or rst == 0:
    log_hist = "hist_registered.log"
    msg_rst = ""
    if rst == 2:
        msg_rst = "DB接続エラー"
    elif rst == 3:
        msg_rst = "tempテーブル作成エラー"
    elif rst == 4:
        msg_rst = "tempテーブルINSERTエラー"
    elif rst == 5:
        msg_rst = "MERGEエラー"

    with open(path_join(root_dir, log_hist) , 'a') as file:
        # 0:処理タイプ, 1:実行プログラム名, 2:装置ID, 3:装置名 4:ユニットNo, 5:ユニット名, 6:logNo, 7:登録データ種類, 8:登録先テーブル名(FTPは空白) 9:処理完了時間, 10:エラーNo, 11:エラー詳細, 12:最終データ発生日時
        file.write('"' + ProcType + '"' + ',' +
                   '"' + os.path.basename(__file__) + '"' + ',' +
                   '"' + eqp_id + '"' + ',' +
                   '"' + eqp_name + '"' + ',' +
                   '"' + unit_num + '"' + ',' +
                   '"' + unit_name + '"' + ',' +
                   '"' + str(log_num) + '"' + ',' + 
                   '"' + Dtype + '"' + ',' + 
                   '"' + target_table_base + '"' + ',' + 
                   '"' + dt_en.strftime('%Y-%m-%d %H:%M:%S') + '"' + ',' + 
                   '"' + str(rst) + '"' + ',' + 
                   '"' + msg_rst + '"' + ',' + 
                   '"' + measured_dt_latest + '"' + linesep)
        
if rst != 1:
    logger.info("************" + eqp_id + " " + unit_num + " " + unit_name + " " + Dtype + ("" if log_num == "" else ("_" + str(log_num))) + "の書き込み END*******************")

